In [1]:
import base64
import glob
import os
import pickle

import cv2
import numpy as np
import requests

In [2]:
# อ่านไฟล์
test_part = 'D:\AI66\\teaincarmodel\\test' 
part = 'D:\AI66\\teaincarmodel\\train'
# print(train_part)

In [3]:
test_list = os.listdir(test_part)  # รายการของไฟล์ทั้งหมดในโฟลเดอร์
train_list = os.listdir(part)
# print(test_list)

In [4]:
def setBrandNumber(brand):
    index = test_list.index(brand)  
    return index
# print(setBrandNumber('Audi'))

In [5]:
def setBrandNumber(brand):
    index = test_list.index(brand)  
    return index
# print(setBrandNumber('Audi'))

In [6]:
url = "http://localhost:5000/api/gethog"
def imgCarbase64(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url, json={"img":data})
    return response.json() 

In [7]:
# img = cv2.imread('D:\AI66\\teaincarmodel\\train\Audi\\1.jpg')
# print(imgCarbase64(img))

In [8]:
#อ่านข้อมูลภาพมาไว้ใน list (X)
def readData(part):
    response = []
    for folder in os.listdir(part):
        for img_name in os.listdir(part+'\\'+folder):
            img_path = part+'\\'+folder+'\\'+img_name
            img = cv2.imread(img_path) # การแปลงรูปภาพเป็นตัวเลข
            res = imgCarbase64(img) #การเรียนใช้ Methot api เพื่อทำการแปลง img เป็น hog vector
            hog = list(res["hog"])
            hog.append(train_list.index(folder)) #การนำตัวเลขมาต่อกัน
            response.append(hog)  # ค่าทีได้จะถูกเก็บไว้ใน list
    return response        

In [9]:
testDatatrain =  readData(part)
testDatatest  =  readData(test_part)

In [10]:
print(len(testDatatest))

813


In [11]:
# ฟังก์ชันการบันทึกข้อมูล บัททึกข้อมูลเป็นไฟล์ pkl 
def savePkl(filename ,cars):
        
    write_path = filename + ".pkl"
    pickle.dump(cars, open(write_path,"wb"))
    print("data preparation is done")

# load data to pkl function    
def loadPkl(filename):
    dataset = pickle.load(open(filename + '.pkl','rb'))
    return dataset

In [12]:
# เรียกใช้ ฟังก์ชัน บันทึกไฟล์ Pkl โดยส่ง path ของโฟลเดอร์ train ไป
savePkl('train_cars',testDatatrain)


data preparation is done


In [13]:
savePkl('test_cars',testDatatest)

data preparation is done


In [14]:
# เรียกใช้ function loadPkl โดยส่งชื่อไฟล์ไป
dataset_train = loadPkl('train_cars')
print("Data train : ",len(dataset_train))
dataset_test = loadPkl('test_cars')
print("Data test : ",len(dataset_test))

Data train :  3352
Data test :  813


In [15]:
# แปลง list ข้อมูล train เป็น array
train_arr = np.array(dataset_train)
x_train = train_arr[:,0:-1]
y_train = train_arr[:,-1]

In [16]:
# แปลง list ข้อมูล test เป็น array
test_arr = np.array(dataset_test)
x_test = test_arr[:,0:-1]
y_test = test_arr[:,-1]

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [19]:
# เทรนข้อมูลด้วย Decision Tree
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [20]:
# ทดสอบประสิทธิภาพของข้อมูล
y_pred = clf.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 48.708487084870846
Confusion Matrix : 
 [[ 81  19  14  18  21  24  22]
 [  8  39   4   4   3   5   4]
 [ 13   5  27   8   1   6  15]
 [ 24   3  11  14   5   6  11]
 [ 10  12   5   8  49   2  16]
 [ 12   3   0   7   9  66   9]
 [ 26  10   4  10  13   7 120]]


In [21]:
# บันทึก model file ด้วย pickle
path_model ='carbrandmodel.pkl'
pickle.dump(clf, open(path_model,'wb'))